In [1]:
# importacoes
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import openpyxl
import time
import pandas as pd

In [2]:
# definindo lista de empresas:
lista_br = pd.read_excel('lista_hospitais_na_bolsa.xlsx')
tickers = lista_br['id_acao']
tickers

0    AALR3
1    DASA3
2    FLRY3
3    HAPV3
4    KRSA3
5    MATD3
6    ODPV3
7    ONCO3
8    QUAL3
9    RDOR3
Name: id_acao, dtype: object

In [3]:
# - 1 utilização do webdriver

browser = webdriver.Chrome()
time.sleep(2)
browser.get('https://statusinvest.com.br/acoes/')
time.sleep(15)


In [4]:
# remover propagandas
ad_close = browser.find_element(By.XPATH, '//button[@class="btn-close"]')
ad_close.click()
time.sleep(1)

In [5]:
# raspar os históricos de FLUXO DE CAIXA
with pd.ExcelWriter('raspagem_tabela_fluxo.xlsx', engine='openpyxl') as writer:
    for ticker in tickers:
        #Acessando conteúdo de cada empresa
        lupa = browser.find_element(By.XPATH, '//a[@class="main-search" and @title="Busca por nome de empresas, FIIs, índices e muito mais"]')
        lupa.click()
        input =  browser.find_element(By.XPATH, '//input[@class="Typeahead-input input tt-input"]')
        input.send_keys(ticker)
        time.sleep(3)
        input.send_keys(Keys.ARROW_DOWN)
        input.send_keys(Keys.ENTER)
        time.sleep(3)
        
        # SCROLLS para carregar página
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # clicar em TODOS ver mais para html carregar
        #detecta quantos ver mais tem:
        ver_mais = browser.find_elements(By.XPATH, '//button[@title="Ver tudo"]')
        len(ver_mais) # tem 3 ou 4 'ver mais'
        time.sleep(1)

        # gera loop para clicar em todos botoes 'ver mais':
        for botao in range(len(ver_mais)):
            # Rolar até o ver mais
            browser.execute_script("arguments[0].scrollIntoView(true);", ver_mais[botao])
            browser.execute_script("window.scrollBy(0, -400);")
            time.sleep(1)

            #clicar no ver mais
            ver_mais[botao].click()
            time.sleep(2)

        # simplificar tabelas (eliminar análises horizontais)
        # encontrar input que quero clicar
        # tem um botao verde para cada uma das tabelas
        botao_verde = browser.find_elements(By.XPATH, '//div[@data-tooltip="Apresentar/esconder colunas do grid"]')

        for botao in range(len(botao_verde)):
            browser.execute_script("arguments[0].scrollIntoView(true);", botao_verde[botao])
            time.sleep(1)
            browser.execute_script("window.scrollBy(0, -300);")
            time.sleep(1)
            input_bv1 = botao_verde[botao].find_element(By.TAG_NAME, 'input')
            input_bv1.click()
            #encontrar checkbox e clicar
            ul_bv = botao_verde[botao].find_element(By.TAG_NAME, 'ul')
            li = ul_bv.find_elements(By.TAG_NAME, 'li')
            for j in range(len(li)):
                if (li[j].text == 'AH'):
                    checkbox = li[j]
                    checkbox.click()
                    time.sleep(2)
            checkbox.send_keys(Keys.ESCAPE)
            time.sleep(1)
                    
        # divs resultado, fluxo, e balanço
        divs_resultado_e_fluxo = browser.find_elements(By.XPATH, '//div[@class="DRE table-info card card-panel white mb-5"]')

        div_resultado = divs_resultado_e_fluxo[0] # DIV RESULTADO É A PRIMEIRA
        div_resultado_inteira_text = div_resultado.text.split('\n')
        print(len(div_resultado_inteira_text)) # guardar - variável com textos da div com resultados

        div_fluxo = divs_resultado_e_fluxo[1] # DIV FLUXO É A SEGUNDA
        div_fluxo_inteira_text = div_fluxo.text.split('\n')
        print(len(div_fluxo_inteira_text)) # guardar - variável com textos da div com fluxo
        
        time.sleep(2)
        
        # Numero de anos
        divs_th = browser.find_elements(By.XPATH, '//th[@class="text-right DATA timeType-0"]') # o total de divs th com essa classe estão divididos entre as três tabelas com valores financeiros (resultado, fluxo de caixa e balanço patrimial)
        numero_de_anos = len(divs_th)/3
        numero_de_anos = int(numero_de_anos)
        anos = []
        for ano in range(len(divs_th)):
            #print(divs_th[ano].text)
            anos.append(divs_th[ano].text)
        print(anos)
            
        # Criar um conjunto para armazenar anos únicos
        anos_passados = set()

        # Lista para armazenar anos não repetidos
        anos_nao_repetidos = []

        # Percorrer a lista de anos
        for ano in anos:
        # Verificar se o ano já foi visitado
            if ano not in anos_passados:
                anos_nao_repetidos.append(ano)
                anos_passados.add(ano)
            else:
                # Se encontrar um ano repetido, parar o loop
                break

        # Imprimir a lista de anos não repetidos
        print(anos_nao_repetidos) # printa a array com os anos que irão compor o header do dataframe
        numero_de_anos = len(anos_nao_repetidos)
        # print(anos) 
        
        # pegando valores conforme padrão detectado no listão de textos:
        # Receita Líquida
        caixa_liq_atv_oper = []
        indice_inicial = 10
        
        for texto in range(10, 10 + numero_de_anos):
            caixa_liq_atv_oper.append(div_fluxo_inteira_text[texto])
        print(caixa_liq_atv_oper)
        
        #Daqui para baixo é só encontrar o pradrão nas linhas do texto...
        ##################################################
        caixa_gerado_nas_operacoes = []
        for texto in range(10 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos):
            caixa_gerado_nas_operacoes.append(div_fluxo_inteira_text[texto])
        print(caixa_gerado_nas_operacoes)
        
        #time.sleep(2)
        
        lucro_liquido = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos):
            lucro_liquido.append(div_fluxo_inteira_text[texto])
        print(lucro_liquido)
        
       # time.sleep(2)
        
        equivalencia_patrimonial = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos):
            equivalencia_patrimonial.append(div_fluxo_inteira_text[texto])
        print(equivalencia_patrimonial)
        
        #time.sleep(2)
        
        depre_e_amort = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            depre_e_amort.append(div_fluxo_inteira_text[texto])
        print(depre_e_amort)
            
        #time.sleep(2)
        
        var_ativo_pass = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            var_ativo_pass.append(div_fluxo_inteira_text[texto])
        print(var_ativo_pass)

        caixa_liq_atv_invest = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            caixa_liq_atv_invest.append(div_fluxo_inteira_text[texto])
        print(caixa_liq_atv_invest)
        
        #time.sleep(2)
        
        fluxo_de_caixa_livre = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            fluxo_de_caixa_livre.append(div_fluxo_inteira_text[texto])
        print(fluxo_de_caixa_livre)
        
       #time.sleep(2)
        
        caixa_liq_atv_financ = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            caixa_liq_atv_financ.append(div_fluxo_inteira_text[texto])
        print(caixa_liq_atv_financ)
          
        #time.sleep(2)  
        
        var_cambial_de_cx_e_eqiv = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            var_cambial_de_cx_e_eqiv.append(div_fluxo_inteira_text[texto])
        print(var_cambial_de_cx_e_eqiv)
        
        #time.sleep(2)
        
        aum_de_cx_e_equiv = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            aum_de_cx_e_equiv.append(div_fluxo_inteira_text[texto])
        print(aum_de_cx_e_equiv)
         
        #time.sleep(2)   
        
        saldo_inic_de_cx_e_equiv = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            saldo_inic_de_cx_e_equiv.append(div_fluxo_inteira_text[texto])
        print(saldo_inic_de_cx_e_equiv)
        
        #time.sleep(2) 
           
        saldo_final_de_cx_e_equiv = []
        for texto in range(10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 10 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            saldo_final_de_cx_e_equiv.append(div_fluxo_inteira_text[texto])
        print(saldo_final_de_cx_e_equiv)
            
        raspagem = {
            'Ano': anos_nao_repetidos,
            'Caixa Líquido Atividades Operacionais - (R$)': caixa_liq_atv_oper,
            'Caixa Gerado nas Operações - (R$)':caixa_gerado_nas_operacoes,
            'Lucro Líquido - (R$)': lucro_liquido,
            'Equivalência Patrimonial - (R$)': equivalencia_patrimonial,
            'Depreciação e Amortização - (R$)': depre_e_amort,
            'Variações nos Ativos e Passivos - (R$)': var_ativo_pass,
            'Caixa Líquido Atividades de Investimento - (R$)':caixa_liq_atv_invest,
            'Fluxo de Caixa Livre - (R$': fluxo_de_caixa_livre,
            'Caixa Líquido Atividades de Financiamento - (R$)': caixa_liq_atv_financ,
            'Variação Cambial de Caixa e Equivalentes - (R$)':var_cambial_de_cx_e_eqiv,
            'Aumento de Caixa e Equivalentes - (R$)':aum_de_cx_e_equiv,
            'Saldo Inicial de Caixa e Equivalentes - (R$)':saldo_inic_de_cx_e_equiv,
            'Saldo Final de Caixa e Equivalentes - (R$)':saldo_final_de_cx_e_equiv,
        }


        # NOME DA EMPRESA E CNPJ PARA PÔR EM CADA LINHA
        company_description = browser.find_element(By.XPATH, '//div[@class="company-description w-100 w-md-70 ml-md-5"]')
        nome_cnpj = company_description.text.split('\n')
        nome_empresa = nome_cnpj[0]
        nome_empresa = nome_empresa.replace('/', '.')
        cnpj_empresa = nome_cnpj[1]

        # ADICIONANDO NOME E CNPJ AO DATAFRAME
        
        raspagem_df = pd.DataFrame(raspagem)

        numero_linhas = raspagem_df.shape[0]
        # numero_linhas

        empresa = []
        for n in range(numero_linhas):
            empresa.append(nome_empresa)
        # empresa
        raspagem_df.insert(0, 'empresa', empresa)

        cnpj = []
        for n in range(numero_linhas):
            cnpj.append(cnpj_empresa)
        # cnpj

        raspagem_df.insert(1, 'cnpj', cnpj)
        # raspagem_df
        
        
        # raspagem_df['empresa'] = nome_empresa
        # raspagem_df['cnpj'] = cnpj
        display(raspagem_df)
        raspagem_df.to_excel(writer, sheet_name=nome_empresa[:18], header=True, index=False)
        

206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['187,30 M', '149,13 M', '117,86 M', '213,41 M', '161,19 M']
['26,30 M', '140,39 M', '48,77 M', '186,36 M', '162,29 M']
['-219,24 M', '1,99 M', '-92,43 M', '50,07 M', '51,57 M']
['-', '-', '-', '-', '-']
['117,74 M', '119,60 M', '119,18 M', '117,87 M', '82,62 M']
['168,73 M', '16,35 M', '62,33 M', '26,39 M', '-7,69 M']
['-142,50 M', '-105,70 M', '-64,68 M', '-77,05 M', '-116,01 M']
['44,81 M', '43,43 M', '53,19 M', '136,36 M', '45,18 M']
['46,63 M', '-145,21 M', '87,60 M', '-121,22 M', '-63,61 M']
['0,00', '0,00', '0,00', '0,00', '0,00']
['91,43 M', '-101,78 M', '140,79 M', '15,14 M', '-18,43 M']
['127,31 M', '229,09 M', '88,30 M', '73,17 M', '91,60 M']
['218,74 M', '127,31 M', '229,09 M', '88,30 M', '73,17 M']


,empresa,cnpj,Ano,Caixa Líquido Atividades Operacionais - (R$),Caixa Gerado nas Operações - (R$),Lucro Líquido - (R$),Equivalência Patrimonial - (R$),Depreciação e Amortização - (R$),Variações nos Ativos e Passivos - (R$),Caixa Líquido Atividades de Investimento - (R$),Fluxo de Caixa Livre - (R$,Caixa Líquido Atividades de Financiamento - (R$),Variação Cambial de Caixa e Equivalentes - (R$),Aumento de Caixa e Equivalentes - (R$),Saldo Inicial de Caixa e Equivalentes - (R$),Saldo Final de Caixa e Equivalentes - (R$)
0,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2022,"187,30 M","26,30 M","-219,24 M",-,"117,74 M","168,73 M","-142,50 M","44,81 M","46,63 M","0,00","91,43 M","127,31 M","218,74 M"
1,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2021,"149,13 M","140,39 M","1,99 M",-,"119,60 M","16,35 M","-105,70 M","43,43 M","-145,21 M","0,00","-101,78 M","229,09 M","127,31 M"
2,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2020,"117,86 M","48,77 M","-92,43 M",-,"119,18 M","62,33 M","-64,68 M","53,19 M","87,60 M","0,00","140,79 M","88,30 M","229,09 M"
3,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2019,"213,41 M","186,36 M","50,07 M",-,"117,87 M","26,39 M","-77,05 M","136,36 M","-121,22 M","0,00","15,14 M","73,17 M","88,30 M"
4,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2018,"161,19 M","162,29 M","51,57 M",-,"82,62 M","-7,69 M","-116,01 M","45,18 M","-63,61 M","0,00","-18,43 M","91,60 M","73,17 M"


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['1.339,68 M', '1.298,18 M', '766,50 M', '699,37 M', '530,33 M']
['2.506,96 M', '1.658,49 M', '881,88 M', '1.139,11 M', '816,12 M']
['-387,77 M', '-216,94 M', '-147,74 M', '130,22 M', '185,31 M']
['-', '-', '-', '-', '-']
['1.111,52 M', '929,76 M', '765,59 M', '455,65 M', '259,80 M']
['-1.085,60 M', '-282,47 M', '-81,99 M', '-419,56 M', '-273,70 M']
['-1.599,80 M', '-4.951,72 M', '-941,20 M', '-518,46 M', '-925,72 M']
['-260,12 M', '-3.653,54 M', '-174,70 M', '180,91 M', '-395,39 M']
['402,08 M', '4.042,96 M', '360,50 M', '-157,87 M', '428,49 M']
['0,00', '0,00', '0,00', '0,00', '0,00']
['141,97 M', '389,42 M', '185,80 M', '23,04 M', '33,10 M']
['1.143,03 M', '753,61 M', '567,81 M', '544,77 M', '511,67 M']
['1.284,99 M', '1.143,03 M', '753,61 M', '567,81 M', '544,77 M']


,empresa,cnpj,Ano,Caixa Líquido Atividades Operacionais - (R$),Caixa Gerado nas Operações - (R$),Lucro Líquido - (R$),Equivalência Patrimonial - (R$),Depreciação e Amortização - (R$),Variações nos Ativos e Passivos - (R$),Caixa Líquido Atividades de Investimento - (R$),Fluxo de Caixa Livre - (R$,Caixa Líquido Atividades de Financiamento - (R$),Variação Cambial de Caixa e Equivalentes - (R$),Aumento de Caixa e Equivalentes - (R$),Saldo Inicial de Caixa e Equivalentes - (R$),Saldo Final de Caixa e Equivalentes - (R$)
0,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2022,"1.339,68 M","2.506,96 M","-387,77 M",-,"1.111,52 M","-1.085,60 M","-1.599,80 M","-260,12 M","402,08 M","0,00","141,97 M","1.143,03 M","1.284,99 M"
1,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2021,"1.298,18 M","1.658,49 M","-216,94 M",-,"929,76 M","-282,47 M","-4.951,72 M","-3.653,54 M","4.042,96 M","0,00","389,42 M","753,61 M","1.143,03 M"
2,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2020,"766,50 M","881,88 M","-147,74 M",-,"765,59 M","-81,99 M","-941,20 M","-174,70 M","360,50 M","0,00","185,80 M","567,81 M","753,61 M"
3,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2019,"699,37 M","1.139,11 M","130,22 M",-,"455,65 M","-419,56 M","-518,46 M","180,91 M","-157,87 M","0,00","23,04 M","544,77 M","567,81 M"
4,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2018,"530,33 M","816,12 M","185,31 M",-,"259,80 M","-273,70 M","-925,72 M","-395,39 M","428,49 M","0,00","33,10 M","511,67 M","544,77 M"


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['987,13 M', '1.018,67 M', '647,14 M', '732,32 M', '701,49 M']
['1.380,12 M', '1.168,71 M', '916,59 M', '983,33 M', '799,65 M']
['308,73 M', '351,49 M', '256,96 M', '312,32 M', '331,59 M']
['-0,60 M', '-0,55 M', '0,33 M', '0,33 M', '-0,90 M']
['455,34 M', '389,28 M', '358,47 M', '320,46 M', '189,29 M']
['-203,85 M', '-7,30 M', '-127,66 M', '-143,95 M', '-8,11 M']
['-1.384,47 M', '-591,36 M', '-507,18 M', '-542,63 M', '-805,73 M']
['-397,34 M', '427,31 M', '139,96 M', '189,69 M', '-104,25 M']
['380,87 M', '-421,77 M', '-120,74 M', '-235,96 M', '-177,88 M']
['0,00', '0,00', '0,00', '0,00', '0,00']
['-16,47 M', '5,54 M', '19,22 M', '-46,27 M', '-282,12 M']
['33,72 M', '28,18 M', '8,97 M', '55,23 M', '337,35 M']
['17,26 M', '33,72 M', '28,18 M', '8,97 M', '55,23 M']


...

,empresa,cnpj,Ano,Caixa Líquido Atividades Operacionais - (R$),Caixa Gerado nas Operações - (R$),Lucro Líquido - (R$),Equivalência Patrimonial - (R$),Depreciação e Amortização - (R$),Variações nos Ativos e Passivos - (R$),Caixa Líquido Atividades de Investimento - (R$),Fluxo de Caixa Livre - (R$,Caixa Líquido Atividades de Financiamento - (R$),Variação Cambial de Caixa e Equivalentes - (R$),Aumento de Caixa e Equivalentes - (R$),Saldo Inicial de Caixa e Equivalentes - (R$),Saldo Final de Caixa e Equivalentes - (R$)
0,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2022,"-271,64 M","650,73 M","113,77 M","0,54 M","175,15 M","-513,39 M","-408,72 M","-680,37 M","407,08 M","-0,35 M","-273,63 M","536,84 M","263,21 M"
1,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2021,"-9,28 M","370,29 M","-22,42 M","0,00","126,97 M","-217,29 M","-2.029,91 M","-2.039,19 M","1.834,72 M","0,42 M","-204,05 M","740,89 M","536,84 M"
2,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2020,"185,57 M","212,86 M","-121,34 M","0,00","106,81 M","72,00 M","-355,13 M","-169,57 M","463,11 M","0,00","293,55 M","447,35 M","740,89 M"


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['564,04 M', '807,93 M', '851,27 M', '870,99 M', '779,31 M']
['917,54 M', '1.055,26 M', '1.057,10 M', '1.153,49 M', '1.007,28 M']
['146,73 M', '554,35 M', '552,29 M', '567,82 M', '628,11 M']
['4,89 M', '2,64 M', '0,00', '0,00', '0,00']
['415,42 M', '362,84 M', '367,48 M', '380,74 M', '323,49 M']
['40,86 M', '-91,59 M', '31,63 M', '-40,24 M', '-7,80 M']
['-535,66 M', '-807,51 M', '-564,27 M', '-131,41 M', '-332,88 M']
['28,38 M', '0,42 M', '287,00 M', '739,58 M', '446,43 M']
['-100,71 M', '30,27 M', '-69,28 M', '-675,98 M', '-455,72 M']
['0,00', '0,00', '0,00', '0,00', '0,00']
['-72,33 M', '30,70 M', '217,72 M', '63,60 M', '-9,29 M']
['449,46 M', '418,77 M', '201,05 M', '137,45 M', '146,73 M']
['377,13 M', '449,46 M', '418,77 M', '201,05 M', '137,45 M']


,empresa,cnpj,Ano,Caixa Líquido Atividades Operacionais - (R$),Caixa Gerado nas Operações - (R$),Lucro Líquido - (R$),Equivalência Patrimonial - (R$),Depreciação e Amortização - (R$),Variações nos Ativos e Passivos - (R$),Caixa Líquido Atividades de Investimento - (R$),Fluxo de Caixa Livre - (R$,Caixa Líquido Atividades de Financiamento - (R$),Variação Cambial de Caixa e Equivalentes - (R$),Aumento de Caixa e Equivalentes - (R$),Saldo Inicial de Caixa e Equivalentes - (R$),Saldo Final de Caixa e Equivalentes - (R$)
0,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2022,"564,04 M","917,54 M","146,73 M","4,89 M","415,42 M","40,86 M","-535,66 M","28,38 M","-100,71 M","0,00","-72,33 M","449,46 M","377,13 M"
1,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2021,"807,93 M","1.055,26 M","554,35 M","2,64 M","362,84 M","-91,59 M","-807,51 M","0,42 M","30,27 M","0,00","30,70 M","418,77 M","449,46 M"
2,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2020,"851,27 M","1.057,10 M","552,29 M","0,00","367,48 M","31,63 M","-564,27 M","287,00 M","-69,28 M","0,00","217,72 M","201,05 M","418,77 M"
3,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2019,"870,99 M","1.153,49 M","567,82 M","0,00","380,74 M","-40,24 M","-131,41 M","739,58 M","-675,98 M","0,00","63,60 M","137,45 M","201,05 M"
4,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2018,"779,31 M","1.007,28 M","628,11 M","0,00","323,49 M","-7,80 M","-332,88 M","446,43 M","-455,72 M","0,00","-9,29 M","146,73 M","137,45 M"


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['1.318,30 M', '128,27 M', '-76,95 M', '781,97 M', '788,82 M']
['6.390,04 M', '5.682,52 M', '3.299,21 M', '4.292,30 M', '3.151,49 M']
['1.325,26 M', '2.032,62 M', '483,46 M', '1.698,96 M', '1.645,38 M']
['-', '-', '-', '-', '-']
['1.404,14 M', '1.228,67 M', '844,53 M', '708,80 M', '393,27 M']
['-1.793,54 M', '-3.785,54 M', '-2.011,45 M', '-2.247,65 M', '-1.447,73 M']
['-2.969,20 M', '-1.651,88 M', '-13.388,25 M', '-4.109,21 M', '-2.757,89 M']
['-1.650,90 M', '-1.523,62 M', '-13.465,20 M', '-3.327,24 M', '-1.969,07 M']
['2.636,08 M', '1.501,35 M', '13.543,40 M', '3.203,91 M', '2.020,70 M']
['0,00', '0,00', '0,00', '0,00', '0,00']
['985,18 M', '-22,26 M', '78,20 M', '-123,32 M', '51,63 M']
['124,62 M', '146,88 M', '68,68 M', '192,01 M', '140,38 M']
['1.109,80 M', '124,62 M', '146,88 M', '68,68 M', '192,01 M']


,empresa,cnpj,Ano,Caixa Líquido Atividades Operacionais - (R$),Caixa Gerado nas Operações - (R$),Lucro Líquido - (R$),Equivalência Patrimonial - (R$),Depreciação e Amortização - (R$),Variações nos Ativos e Passivos - (R$),Caixa Líquido Atividades de Investimento - (R$),Fluxo de Caixa Livre - (R$,Caixa Líquido Atividades de Financiamento - (R$),Variação Cambial de Caixa e Equivalentes - (R$),Aumento de Caixa e Equivalentes - (R$),Saldo Inicial de Caixa e Equivalentes - (R$),Saldo Final de Caixa e Equivalentes - (R$)
0,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2022,"1.318,30 M","6.390,04 M","1.325,26 M",-,"1.404,14 M","-1.793,54 M","-2.969,20 M","-1.650,90 M","2.636,08 M","0,00","985,18 M","124,62 M","1.109,80 M"
1,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2021,"128,27 M","5.682,52 M","2.032,62 M",-,"1.228,67 M","-3.785,54 M","-1.651,88 M","-1.523,62 M","1.501,35 M","0,00","-22,26 M","146,88 M","124,62 M"
2,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2020,"-76,95 M","3.299,21 M","483,46 M",-,"844,53 M","-2.011,45 M","-13.388,25 M","-13.465,20 M","13.543,40 M","0,00","78,20 M","68,68 M","146,88 M"
3,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2019,"781,97 M","4.292,30 M","1.698,96 M",-,"708,80 M","-2.247,65 M","-4.109,21 M","-3.327,24 M","3.203,91 M","0,00","-123,32 M","192,01 M","68,68 M"
4,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2018,"788,82 M","3.151,49 M","1.645,38 M",-,"393,27 M","-1.447,73 M","-2.757,89 M","-1.969,07 M","2.020,70 M","0,00","51,63 M","140,38 M","192,01 M"
